#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Word Count Lab: Building a word count application

This lab will build on the techniques covered in the Spark tutorial to develop a simple word count application.  The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data.  In this lab, we will write code that calculates the most common words in the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) retrieved from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page).

This could also be scaled to find the most common words in Wikipedia.

## During this lab we will cover:
* *Part 1:* Creating a base RDD and pair RDDs
* *Part 2:* Counting with pair RDDs
* *Part 3:* Finding unique words and a mean value
* *Part 4:* Apply word count to a file
* *Appendix A:* Submitting your exercises to the Autograder

> Note that for reference, you can look up the details of the relevant methods in:
> * [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

In [0]:
labVersion = 'cs120x-lab1b-1.0.0'

## Part 1: Creating a base RDD and pair RDDs

In this part of the lab, we will explore creating a base RDD with `parallelize` and using pair RDDs to count words.

### (1a) Create a base RDD
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

In [0]:
wordsList = ['cat', 'rat', 'elephent', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList)
print(type(wordsList))
print(type(wordsRDD))


<class 'list'>
<class 'pyspark.rdd.RDD'>


In [0]:
wordsRDD

Out[5]: ParallelCollectionRDD[1] at readRDDFromInputStream at PythonRDD.scala:435

In [0]:
# thu thập tất cả các phần tử trong RDD
wordsRDD.collect()

Out[6]: ['cat', 'rat', 'elephent', 'rat', 'cat']

In [0]:
# loại bỏ các phần tử trùng lặp trong RDD
wordsRDD.distinct().collect()

Out[7]: ['rat', 'cat', 'elephent']

In [0]:
wordsRDD.distinct().count()

Out[8]: 3

In [0]:
# trả về 2 phần tử đầu tiên của RDD
wordsRDD.take(2)

Out[10]: ['cat', 'rat']

In [0]:
import pandas as pd
df = pd.DataFrame({"student_id" : [10, 54, 65],
                  "full_name": ['Bob', 'Alice', 'Calvin'],
                  "score": [9.5, 8.0, 7]})
df.head()

,student_id,full_name,score
0,10,Bob,9.5
1,54,Alice,8.0
2,65,Calvin,7.0


In [0]:
def add_score(r):
    return r["score"] + 0.5
df["score"] = df.apply(add_score, axis=1)
df.head()

,student_id,full_name,score
0,10,Bob,11.0
1,54,Alice,9.5
2,65,Calvin,8.5


### (1b) Pluralize and test

Let's use a `map()` transformation to add the letter 's' to each string in the base RDD we just created. We'll define a Python function that returns the word with an 's' at the end of the word.  Please replace `<FILL IN>` with your solution.  If you have trouble, the next cell has the solution.  After you have defined `makePlural` you can run the third cell which contains a test.  If you implementation is correct it will print `1 test passed`.

This is the general form that exercises will take, except that no example solution will be provided.  Exercises will include an explanation of what is expected, followed by code cells where one cell will have one or more `<FILL IN>` sections.  The cell that needs to be modified will have `# TODO: Replace <FILL IN> with appropriate code` on its first line.  Once the `<FILL IN>` sections are updated and the code is run, the test cell can then be run to verify the correctness of your solution.  The last code cell before the next markdown section will contain the tests.

In [0]:
def makePlural(word):
    return word + 's'
print(makePlural('cat'))

cats


In [0]:
!pip install databricks_test_helper

  Created wheel for databricks-test-helper: filename=databricks_test_helper-1.0.0-py3-none-any.whl size=2065 sha256=5d6fe6f9bb420cb5dc4b83f311135be96f5e2bda1b307ddb51cd82800969bd64
  Stored in directory: /root/.cache/pip/wheels/14/d9/f3/77edd1baa329fcfacdee3cbe49a31f18a892a1f868873da4be
Successfully built databricks-test-helper
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-cbb30321-f685-4843-8613-a15b24d3a667/bin/python -m pip install --upgrade pip' command.


In [0]:
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
# from databricks_test_helper import Test
# #TEST Pluralize and test (1b)
# Test.assertEquals(makePlural('rat'), 'rats', 'incorrect result: makePlural does not add an s')
assert makePlural('rat') == 'rats'
assert makePlural('elephant') == 'elephants', 'Assertion failed'

### (1c) Apply `makePlural` to the base RDD

Now pass each item in the base RDD into a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation that applies the `makePlural()` function to each element. And then call the [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) action to see the transformed RDD.

In [0]:
wordsRDD.collect()

Out[23]: ['cat', 'rat', 'elephent', 'rat', 'cat']

In [0]:
pluralRDD = wordsRDD.map(makePlural)
print(pluralRDD.collect())

['cats', 'rats', 'elephents', 'rats', 'cats']


In [0]:
rdd_test = sc.parallelize(['house', 'school', 'dog'])
assert rdd_test.map(makePlural).collect() == ['houses', 'schools', 'dogs'], 'makePlural() error!!'

### (1d) Pass a `lambda` function to `map`

Let's create the same RDD using a `lambda` function.

In [0]:
pluralLambdaRDD = wordsRDD.map(lambda x : x + 's')
pluralLambdaRDD.collect()

Out[26]: ['cats', 'rats', 'elephents', 'rats', 'cats']

In [0]:
assert pluralLambdaRDD.collect() == ['cats', 'rats', 'elephents', 'rats', 'cats'], 'incorrect values for pluralLambdaRDD (1d)'

### (1e) Length of each word

Now use `map()` and a `lambda` function to return the number of characters in each word.  We'll `collect` this result directly into a variable.

In [0]:
pluralLengths = (pluralRDD.map(lambda x : len(x)).collect())
print(pluralLengths)

[4, 4, 9, 4, 4]


In [0]:
# TEST Length of each word (1e)
assert pluralLengths == [4, 4, 9, 4, 4],\
            'incorrect values for pluralLengths'

### (1f) Pair RDDs

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple `(k, v)` where `k` is the key and `v` is the value. In this example, we will create a pair consisting of `('<word>', 1)` for each word element in the RDD.
We can create the pair RDD using the `map()` transformation with a `lambda()` function to create a new RDD.

In [0]:
wordPairs = wordsRDD.map(lambda x : (x, 1))
wordPairs.collect()

Out[39]: [('cat', 1), ('rat', 1), ('elephent', 1), ('rat', 1), ('cat', 1)]

In [0]:
test = (3,)
test

Out[44]: (3,)

In [0]:
# TEST Pair RDDs (1f)
assert wordPairs.collect() ==\
                  [('cat', 1), ('rat', 1), ('elephent', 1), ('rat', 1), ('cat', 1)],\
                  'incorrect value for wordPairs'

## Part 2: Counting with pair RDDs

Now, let's count the number of times a particular word appears in the RDD. There are multiple ways to perform the counting, but some are much less efficient than others.

A naive approach would be to `collect()` all of the elements and count them in the driver program. While this approach could work for small datasets, we want an approach that will work for any size dataset including terabyte- or petabyte-sized datasets. In addition, performing all of the work in the driver program is slower than performing it in parallel in the workers. For these reasons, we will use data parallel operations.

### (2a) `groupByKey()` approach
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions.

There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.

Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`.

In [0]:
wordPairs.collect()

Out[45]: [('cat', 1), ('rat', 1), ('elephent', 1), ('rat', 1), ('cat', 1)]

In [0]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print(type(value))
    print('{0}: {1}'.format(key, list(value)))

<class 'pyspark.resultiterable.ResultIterable'>
rat: [1, 1]
<class 'pyspark.resultiterable.ResultIterable'>
cat: [1, 1]
<class 'pyspark.resultiterable.ResultIterable'>
elephent: [1]


In [0]:
wordsGrouped.collect()

Out[49]: [('rat', <pyspark.resultiterable.ResultIterable at 0x7f62540ea1f0>),
 ('cat', <pyspark.resultiterable.ResultIterable at 0x7f62540ea070>),
 ('elephent', <pyspark.resultiterable.ResultIterable at 0x7f62540ea0a0>)]

In [0]:
assert sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()) == \
    [("cat", [1, 1]), ("elephent", [1]), ("rat", [1, 1])],\
        'incorrect value for wordsGrouped'

### (2b) Use `groupByKey()` to obtain the counts

Using the `groupByKey()` transformation creates an RDD containing 3 elements, each of which is a pair of a word and a Python iterator.

Now sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

# wordsGrouped
* cat: [1, 1]
* elephant: [1]
* rat: [1, 1]

In [0]:
wordCountsGrouped = wordsGrouped.mapValues(lambda y: sum(y))
print(wordCountsGrouped.collect())

[('rat', 2), ('cat', 2), ('elephent', 1)]


In [0]:
# TEST Use groupByKey() to obtain the counts (2b)
sorted(wordCountsGrouped.collect()) ==\
                  [('cat', 2), ('elephent', 1), ('rat', 2)],\
                  'incorrect value for wordCountsGrouped'

Out[53]: (True, 'incorrect value for wordCountsGrouped')

###(2c) Counting using `reduceByKey`

A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

In [0]:
wordCounts = wordPairs.reduceByKey(lambda x,y : x+y)
print(wordCounts.collect())

[('rat', 2), ('cat', 2), ('elephent', 1)]


In [0]:
rdd_words_2 = sc.parallelize([('rat', 3), ('elephant', 1), ('dog', 1), ('cat', 2)])
rdd_words_2.collect()

Out[56]: [('rat', 3), ('elephant', 1), ('dog', 1), ('cat', 2)]

In [0]:
wordsRDD = sc.parallelize(rdd_words_2)
wordsRDD

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3227210225190488>:1
----> 1 wordsRDD = sc.parallelize(rdd_words_2)
      2 wordsRDD

File /databricks/spark/python/pyspark/context.py:866, in SparkContext.parallelize(self, c, numSlices)
    864 # Make sure we distribute data evenly if it's smaller than self.batchSize
    865 if "__len__" not in dir(c):
--> 866     c = list(c)  # Make it a list so we can compute its length
    867 serializer = self.init_batched_serializer(c, numSlices)
    869 if self._conf.get(
    870     self._jvm.PythonSecurityUtils.SPARK_PYTHON_TO_JVM_TRUSTED_DATA_TRANSFER_ENABLED()
    871 ):
    872     # This is a bit of a hack: we call into the SparkSession API from the SparkContext API,
    873     # since the SparkSession is where the JVM knows the Python username

TypeError: 'RDD' object is not iterable

In [0]:
rdd_word_pairs_2 = rdd_words_2.map(lambda x: (x, 1))
rdd_word_pairs_2.collect()
rdd_word_count = rdd_word_pairs_2.reduceByKey(lambda x, y: x+y)
rdd_word_count.collect()


Out[62]: [(('cat', 2), 1), (('rat', 3), 1), (('dog', 1), 1), (('elephant', 1), 1)]

### (2d) All together

The expert version of the code performs the `map()` to pair RDD, `reduceByKey()` transformation, and `collect` in one statement.

In [0]:
wordCountsCollected = (wordsRDD.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).collect())
print(wordCountsCollected)

[('rat', 2), ('cat', 2), ('elephent', 1)]


## Part 3: Finding unique words and a mean value

### (3a) Unique words

Calculate the number of unique words in `wordsRDD`.  You can use other RDDs that you have already created to make this easier.

In [0]:
wordsRDD.distinct().collect()

Out[64]: ['rat', 'cat', 'elephent']

### (3b) Mean using `reduce`

Find the mean number of words per unique word in `wordCounts`.

Use a `reduce()` action to sum the counts in `wordCounts` and then divide by the number of unique words.  First `map()` the pair RDD `wordCounts`, which consists of (key, value) pairs, to an RDD of values.

In [0]:
wordCounts.collect()

Out[65]: [('rat', 2), ('cat', 2), ('elephent', 1)]

In [0]:
scores = sc.parallelize([('Toan', 9), ("Liu", 8), ("Alex", 7), ("Ivan", 8)])
from operator import add
t = (scores.map(lambda x : x[1]).reduce(add))
s = t / scores.count()
s

Out[68]: 8.0

## Part 4: Apply word count to a file

In this section we will finish developing our word count application.  We'll have to build the `wordCount` function, deal with real world problems like capitalization and punctuation, load in our data source, and compute the word count on the new data.

### (4a) `wordCount` function

First, define a function for word counting.  You should reuse the techniques that have been covered in earlier parts of this lab.  This function should take in an RDD that is a list of words like `wordsRDD` and return a pair RDD that has all of the words and their associated counts.

In [0]:
def wordCount(wordListRDD):
    return wordListRDD\
        .map(lambda x: (x, 1))\
        .reduceByKey(lambda x, y: x+y)
print(wordCount(wordsRDD).collect())

[('rat', 2), ('cat', 2), ('elephent', 1)]


### (4b) Capitalization and punctuation

Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.

Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.
If you are unfamiliar with regular expressions, you may want to review [this tutorial](https://developers.google.com/edu/python/regular-expressions) from Google.  Also, [this website](https://regex101.com/#python) is  a great resource for debugging your regular expression.

In [0]:
import re
def removePunctuation(text):
  t1 = text.lower()
  t2 = re.sub(r'[^0-9a-z\s]', "", t1)
  t3 = t2.strip()
  return t3
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(' *      Remove punctuation then spaces  * '))

hi you
no underscore
remove punctuation then spaces


### (4c) Load a text file

For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lower case.  Since the file is large we use `take(15)`, so that we only print 15 lines.

In [0]:
import os.path
fileName = "dbfs:/" + os.path.join('databricks-datasets', 'cs100', 'lab1', 'data-001', 'shakespeare.txt')
                # to (line, lineNum)
                # to 'lineNum: line'

shakespeareRDD = sc.textFile(fileName, 8).map(removePunctuation)
print(shakespeareRDD.take(15))
print('\n'.join(shakespeareRDD\
                .zipWithIndex().map(lambda x: '{0}: {1}'.format(x[1], x[0]))\
                .take(15)))

['1609', '', 'the sonnets', '', 'by william shakespeare', '', '', '', '1', 'from fairest creatures we desire increase', 'that thereby beautys rose might never die', 'but as the riper should by time decease', 'his tender heir might bear his memory', 'but thou contracted to thine own bright eyes', 'feedst thy lights flame with selfsubstantial fuel']
0: 1609
1: 
2: the sonnets
3: 
4: by william shakespeare
5: 
6: 
7: 
8: 1
9: from fairest creatures we desire increase
10: that thereby beautys rose might never die
11: but as the riper should by time decease
12: his tender heir might bear his memory
13: but thou contracted to thine own bright eyes
14: feedst thy lights flame with selfsubstantial fuel


In [0]:
import os.path
fileName = "dbfs:/" + os.path.join('databricks-datasets', 'cs100', 'lab1', 'data-001', 'shakespeare.txt')


test = sc.textFile(fileName, 8)
test.count()

Out[74]: 122395

### (4d) Words from lines

Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces. ** Performed in (4d). **
  + The second issue is we need to filter out empty lines. ** Performed in (4e). **

Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/2/library/string.html#string.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

> Note:
> * Do not use the default implemenation of `split()`, but pass in a separator value.  For example, to split `line` by commas you would use `line.split(',')`.

In [0]:
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda x: x.split(" "))
shakespeareWordCount = shakespeareWordsRDD.count()
print(shakespeareWordsRDD.top(10))
print(shakespeareWordCount)

['zwaggerd', 'zounds', 'zounds', 'zounds', 'zounds', 'zounds', 'zounds', 'zounds', 'zounds', 'zounds']
927631


### (4e) Remove empty elements 

The next step is to filter out the empty elements.  Remove all entries where the word is `''`.

In [0]:
shakeWordsRDD = shakespeareWordsRDD.filter(lambda x: x!='')
shakeWordCount = shakeWordsRDD.count()
print(shakeWordCount)

882996


### (4f) Count the words

We now have an RDD that is only words.  Next, let's apply the `wordCount()` function to produce a list of word counts. We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pairs, we need a custom sort function that sorts using the value part of the pair.

You'll notice that many of the words are common English words. These are called stopwords. In a later lab, we will see how to eliminate them from the results.
Use the `wordCount()` function and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [0]:
top15WordsAndCounts = wordCount(shakeWordsRDD)\
.sortBy(lambda x: x[1], ascending=False).take(15) 
print('\n'.join(map(lambda x: '{0}: {1}'.format(x[0], x[1]), top15WordsAndCounts)))

the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678
